<a href="https://colab.research.google.com/github/fairml-research/Counterfactual_Fairness/blob/main/Counterfactual_Fairness_CRIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone "https://github.com/fairml-research/Counterfactual_Fairness.git"
import sys, os
os.chdir('Counterfactual_Fairness')

Cloning into 'Counterfactual_Fairness'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 10.65 KiB | 3.55 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
from models.functions import *
from models.adv_vae import ADV_VAE
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

### 1) **IMPORT THE DATA**

In [4]:
from data_loading.datasets import read_dataset

# split into train/test set
#X_train, X_test, y_train, y_test, sensitive, sensitivet = train_test_split(X, y, Z, test_size=0.1, random_state=4)

X_train, y_train, sensitive, X_test, y_test, sensitivet = read_dataset(name='crimes', fold=1)
X_train, y_train, Z_train, X_test, y_test, Z_test, scaler, scale_df, meanYtrain, stdYtrain= Normalize(X_train, y_train, sensitive, X_test, y_test, sensitivet)

### 2) ADVERSARIAL INFERENCE

a) In this part we are going to train a latent space z from X_train, y_train and the sensitive and we will try to recontruct X and Y from the latent space z and the sensitive. In addtion to that there is an adversarial model for mitigating the dependance with the sensitive attribute.

b) In this scenario we have X that is between 0 and 1. Please note that if you want to adapt X for a continuous scenario you need to change this:
 - line 103 in adv_vae.py replace the BCE loss with an MSE one (F.binary_cross_entropy -> F.mse_loss)
 - line 129 in functions.py remove the sigmoid -> (torch.sigmoid(self.fc41(h3)) -> self.fc41(h3)

c) You can see that the betaadv hyerparameter allows to mitigate the dependance with the sensitive (assessed with the HGR at the end of this senction 2). If betaadv =0 the HGR is superior to 0.9 !! and with a lambdaadv it is closer to 0.5

In [5]:
from torch import nn
sizelat=5
class Adversarial(nn.Module):
    def __init__(self):
        super(Adversarial, self).__init__()
        self.adv_fc1 = nn.Linear(sizelat, 32)
        self.adv_fc2 = nn.Linear(32, 16)
        self.adv_fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.adv_fc1(x)
        x = F.tanh(x)
        #x = F.dropout(x, p=0.8)
        x = self.adv_fc2(x)
        x = F.relu(x)
        #x = F.dropout(x, p=0.8)
        x = self.adv_fc3(x)
        #x = torch.sigmoid(x)
        return x
from models.adv_vae import ADV_VAE
#modelVAE=VAE(X_train.shape[1])
from tqdm import trange

batch_size=512
epochs=100
seed=1
log_interval=1
betaX = 1
betaY = 1
#betaKLD = 10
betammd_E =1#1
#betammd_F =0
betaadv=100 #.1
#device="cpu"
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

from tqdm import trange
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from models.functions import VAE
modelVAE=VAE(X_train.shape[1])

ADV_VAE_S0 = ADV_VAE(batch_size, epochs, seed, log_interval, device, nb_features =X_train.shape[1],
                     model = modelVAE, model_adv = Adversarial, sizelat=sizelat,
                     betaX=betaX,betaY=betaY,betammd_E=betammd_E,betaadv=betaadv
                    )
ADV_VAE_S0.fit(X_train, y_train, Z_train)

Bar desc:   1%|          | 1/101 [00:02<04:43,  2.83s/it]

Loss_Y 1.0004787


Bar desc:   6%|▌         | 6/101 [00:04<00:41,  2.30it/s]

Loss_Y 0.9497099


Bar desc:  11%|█         | 11/101 [00:05<00:28,  3.18it/s]

...

Loss_Y 0.1373161


Bar desc:  95%|█████████▌| 96/101 [00:33<00:01,  2.97it/s]

Loss_Y 0.096603304


Bar desc: 100%|██████████| 101/101 [00:35<00:00,  2.82it/s]

Loss_Y 0.10574269
Done


In [14]:
from models.functions import HGR_NN
senstest=torch.tensor(np.expand_dims(Z_test,axis = 1)).float().to(device)
datatest=torch.tensor(X_test.values).float().to(device)
ydatatest= Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).to(device)
recon_X, z, recon_Y, mu, logvar = ADV_VAE_S0.predict(datatest.view(-1, datatest.shape[1]),senstest,ydatatest)
Loss_X = F.binary_cross_entropy(recon_X, datatest.view(-1, datatest.shape[1]), reduction='mean')
Loss_Y = F.mse_loss(recon_Y, ydatatest, reduction='mean')
print("Loss_X TEST", Loss_X.cpu().detach().numpy(), "Loss_Y TEST",Loss_Y.cpu().detach().numpy())
# lambda = 0
HGR_NNP = HGR_NN(Net_HGR(),Net2_HGR(),device, display=False)
print("HGR NN Test",HGR_NNP(senstest , z,200))

Loss_X TEST 0.51497513 Loss_Y TEST 0.10599426
HGR NN Test 0.57030624


In [15]:
from models.functions import HGR_NN
senstrain=torch.tensor(np.expand_dims(Z_train,axis = 1)).float().to(device)
data=torch.tensor(X_train.values).float().to(device)
ydata= Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).to(device)
recon_X, z, recon_Y, mu, logvar = ADV_VAE_S0.predict(data.view(-1, data.shape[1]),senstrain,ydata)
Loss_X = F.binary_cross_entropy(recon_X, data.view(-1, data.shape[1]), reduction='mean')
Loss_Y = F.mse_loss(recon_Y, ydata, reduction='mean')
print("Loss_X", Loss_X.cpu().detach().numpy(), "Loss_Y",Loss_Y.cpu().detach().numpy() )
# lambda = 0
HGR_NNP = HGR_NN(Net_HGR(),Net2_HGR(),device, display=False)
print("HGR NN Train",HGR_NNP(senstrain , z,200))

Loss_X 0.50609195 Loss_Y 0.10484505
HGR NN Train 0.27441216


### 2) **Unfair Predictor Model** (lambdap=0)

In this second part we are now going to create a predictor model that is counterfactual " UnFair " --> the lambdap of penalization is equal to 0 !

Please note that in the third part we are going to make the predictor model Fairer.

In [16]:
from models.cf_predictor import CF_PREDICTOR


#device = torch.device("cpu")
#device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1]+1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
    def forward(self, x, a):
        x = self.fc1(torch.cat([x,a],1))
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x

batch_idx=0
batch_size=256
epochs=1000
seed=1
log_interval=1
lr=0.0001
nb=20
lambdap=0.
CF_PREDICTOR_S0 = CF_PREDICTOR(regressor="mse",batch_size=batch_size, epochs=epochs, seed=seed, log_interval=log_interval, device=device, model = NN, lr=lr, modelVAE= ADV_VAE_S0, nb=nb, lambdap=lambdap)
CF_PREDICTOR_S0.fit(X_train, y_train, Z_train)

Bar desc:   0%|          | 4/1001 [00:00<00:55, 17.93it/s]

MSE estimation : 1.2260033 conterfactual estimation : 1.9581312e-05


Bar desc:  10%|█         | 102/1001 [00:06<01:09, 12.93it/s]

MSE estimation : 0.31259534 conterfactual estimation : 0.32656652


Bar desc:  20%|██        | 204/1001 [00:13<00:48, 16.51it/s]

...

MSE estimation : 0.25653195 conterfactual estimation : 0.57096213


Bar desc:  90%|█████████ | 903/1001 [00:56<00:05, 16.34it/s]

MSE estimation : 0.20654169 conterfactual estimation : 0.5464819


Bar desc: 100%|██████████| 1001/1001 [01:03<00:00, 15.83it/s]

MSE estimation : 0.23729421 conterfactual estimation : 0.49004087


In [17]:
senstest=torch.tensor(np.expand_dims(Z_test,axis = 1)).float().to(device)
datatest=torch.tensor(X_test.values).float().to(device)
ydatatest= Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).to(device)

Yhat_test=CF_PREDICTOR_S0.predict(datatest,senstest)
MSE_Test = F.mse_loss(Yhat_test,ydatatest)
print("MSE TEST:",MSE_Test.cpu().detach().numpy())

MSE TEST: 0.38039482


In [18]:
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

nb = 1000
Uni = np.random.uniform(Z_train.min(),Z_train.max(),nb)
#Unif_X = torch.tensor(np.random.uniform(Z_train.min(),Z_train.max(),Z_test.shape[0]*nb), dtype=torch.float32).to(device)
Unif_X = torch.tensor(torch.tensor(np.expand_dims(Uni,axis=1)).repeat( X_test.shape[0],1), dtype=torch.float32).to(device)
data_X = torch.tensor(np.repeat(X_test.values, nb,0), dtype=torch.float32).to(device)
Y_X    = torch.tensor(np.repeat(y_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_aprime, z_aprime, recon_Y_aprime, mu_aprime, logvar_aprime = ADV_VAE_S0(data_X.view(-1, data_X.shape[1]),Unif_X,Y_X)
Z_train_X = torch.tensor(np.repeat(Z_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_a, z_a, recon_Y_a, mu_a, logvar_a = ADV_VAE_S0.predict(data_X, Z_train_X , Y_X)

predY_a_prime = CF_PREDICTOR_S0.predict(recon_X_aprime,Unif_X).cpu().detach().numpy()
predY_a = CF_PREDICTOR_S0.predict(recon_X_a,Z_train_X.unsqueeze(1)).cpu().detach().numpy()
print('CF Value:  {:9f}'.format(np.mean((predY_a_prime-predY_a)**2),8))

CF Value:   0.703899


<ipython-input-18-8ffb242bdcb1>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Unif_X = torch.tensor(torch.tensor(np.expand_dims(Uni,axis=1)).repeat( X_test.shape[0],1), dtype=torch.float32).to(device)


For calculating the CF we generate 1000 possible counterfactual observations for each instance of X_test. For this, we repeat the X instances for each counterfactual and aggrate with a mean average loss.
We observe in this scenario that the CF is pretty unfair .. we will try to mitigate this in the third part

### 3) **Fair Predictor Model** (lambdap=1)

In this third part we are now going to create a predictor model that is counterfactual Fair.  At each batch and for each instance, we will generate 20 counterfactual observations (nb=20) from the inference model (from the step 1)

In [19]:
from models.cf_predictor import CF_PREDICTOR

#device = torch.device("cpu")
#device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1]+1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
    def forward(self, x, a):
        x = self.fc1(torch.cat([x,a],1))
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x

batch_idx=0
batch_size=256
epochs=1000
seed=1
log_interval=1
lr=0.0001
nb=20
lambdap=1.
CF_PREDICTOR_S0 = CF_PREDICTOR(regressor="mse",batch_size=batch_size, epochs=epochs, seed=seed, log_interval=log_interval, device=device, model = NN, lr=lr, modelVAE= ADV_VAE_S0, nb=nb, lambdap=lambdap)
CF_PREDICTOR_S0.fit(X_train, y_train, Z_train)

Bar desc:   0%|          | 4/1001 [00:00<01:04, 15.56it/s]

MSE estimation : 0.9321294 conterfactual estimation : 4.1591815e-05


Bar desc:  10%|█         | 104/1001 [00:06<00:53, 16.92it/s]

MSE estimation : 0.3252374 conterfactual estimation : 0.027247746


Bar desc:  20%|██        | 204/1001 [00:12<00:46, 17.25it/s]

...

MSE estimation : 0.1791245 conterfactual estimation : 0.0050879666


Bar desc:  90%|█████████ | 904/1001 [00:56<00:05, 17.00it/s]

MSE estimation : 0.22165921 conterfactual estimation : 0.0042373636


Bar desc: 100%|██████████| 1001/1001 [01:02<00:00, 16.08it/s]

MSE estimation : 0.20582287 conterfactual estimation : 0.003362126


In [20]:
senstest=torch.tensor(np.expand_dims(Z_test,axis = 1)).float().to(device)
datatest=torch.tensor(X_test.values).float().to(device)
ydatatest= Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).to(device)

Yhat_test=CF_PREDICTOR_S0.predict(datatest,senstest)
MSE_Test = F.mse_loss(Yhat_test,ydatatest)
print("MSE TEST:",MSE_Test.cpu().detach().numpy())

MSE TEST: 0.40139252


In [21]:
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")

nb = 1000
Uni = np.random.uniform(Z_train.min(),Z_train.max(),nb)
#Unif_X = torch.tensor(np.random.uniform(Z_train.min(),Z_train.max(),Z_test.shape[0]*nb), dtype=torch.float32).to(device)
Unif_X = torch.tensor(torch.tensor(np.expand_dims(Uni,axis=1)).repeat( X_test.shape[0],1), dtype=torch.float32).to(device)
data_X = torch.tensor(np.repeat(X_test.values, nb,0), dtype=torch.float32).to(device)
Y_X    = torch.tensor(np.repeat(y_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_aprime, z_aprime, recon_Y_aprime, mu_aprime, logvar_aprime = ADV_VAE_S0(data_X.view(-1, data_X.shape[1]),Unif_X,Y_X)
Z_train_X = torch.tensor(np.repeat(Z_test.values, nb,0), dtype=torch.float32).to(device)
recon_X_a, z_a, recon_Y_a, mu_a, logvar_a = ADV_VAE_S0.predict(data_X, Z_train_X , Y_X)

predY_a_prime = CF_PREDICTOR_S0.predict(recon_X_aprime,Unif_X).cpu().detach().numpy()
predY_a = CF_PREDICTOR_S0.predict(recon_X_a,Z_train_X.unsqueeze(1)).cpu().detach().numpy()
print('CF Value:  {:9f}'.format(np.mean((predY_a_prime-predY_a)**2),8))

CF Value:   0.003216


<ipython-input-21-8ffb242bdcb1>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Unif_X = torch.tensor(torch.tensor(np.expand_dims(Uni,axis=1)).repeat( X_test.shape[0],1), dtype=torch.float32).to(device)
